In [50]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import datetime 
import requests
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import r2_score

In [51]:
def collect_data(url):
   feeds_ = requests.get(url).json()['feeds']
   feeds_data = pd.DataFrame(feeds_)
   # drop columns which will be unused: Latitude, longitude, gps, battery, pm10 values 
   feeds_data.drop(columns = ['field2','field4','field5','field6','field8','field7','entry_id'],inplace = True)
   #Rename fields correspomnding to pm2.5
   feeds_data.rename(columns ={'field1': 'sensor1_PM2.5','field3':'sensor2_PM2.5','created_at':'timestamp'},inplace = True)
   # converting the date field to timestamp 
   feeds_data['timestamp'] = pd.to_datetime(feeds_data['timestamp'])
   # setting time stamp as the index 
   feeds_data.set_index('timestamp',inplace = True)
   feeds_data = feeds_data.astype({'sensor1_PM2.5':'float','sensor2_PM2.5':'float'}).round(2)
   #feeds_data[['sensor1_PM2.5'],['sensor2_PM2.5']] = pd.to_numeric([['sensor1_PM2.5'],['sensor2_PM2.5']],errors = 'coerce')
   feeds_data = feeds_data.assign(average_PM10 = lambda x : (feeds_data['sensor1_PM2.5'] + feeds_data['sensor2_PM2.5'])/2)
   feeds_data.drop(columns = ['sensor1_PM2.5','sensor2_PM2.5'],inplace = True)
   return feeds_data 
   

In [56]:
def parformance_comparisons(dataframe1,dataframe2):
    combined_dataset = pd.merge(dataframe1,dataframe2,on='TimeStamp')
    mae = mean_absolute_error( dataframe1["value_x"], dataframe2['value_y'])
    rmse = mean_squared_error(dataframe1["value_x"], dataframe2['value_y'])
    rsquared = r2_score(dataframe1["value_x"], dataframe2['value_y'])
    metric = ['mae','rmse','rsquared']
    values= [mae,rmse,rsquared]
    comparison_df = pd.DataFrame(list(zip(metric,values)),columns=['metris','value'])
    return comparison_df
    

In [55]:
# data for analysis using the device AQ_G5133
AQ_G5133 = collect_data('https://thingspeak.com/channels/1962719/feeds.json?start=2023-02-18T00:00:00Z&end=2023-02-20T23:59:59Z&api_key=0IBG2XN6MIRMUMQ17')
AQ_G5133.head(2)

,average_PM10
timestamp,
2023-02-18 00:01:54+00:00,85.440
2023-02-18 00:02:54+00:00,88.755


In [54]:
AQ_G5133.dtypes

average_PM10    float64
dtype: object

In [62]:
df1 = pd.DataFrame({'rkey': ['foo', 'bar', 'baz', 'foo'],
                    'value': [1, 2, 3, 5]})
df2 = pd.DataFrame({'rkey': ['foo', 'bar', 'baz', 'foo'],
                    'value': [5, 6, 7, 8]})
df = pd.merge(df1,df2,on ='rkey')
df

,rkey,value_x,value_y
0,foo,1,5
1,foo,1,8
2,foo,5,5
3,foo,5,8
4,bar,2,6
5,baz,3,7


In [69]:
mae = mean_absolute_error( df["value_x"], df['value_y'])
rmse = mean_squared_error(df["value_x"], df['value_y'])
rsquared = r2_score(df["value_x"], df['value_y'])
lst = ['mae','rmse','rsquared']
lst2= [mae,rmse,rsquared]
results_df = pd.DataFrame(list(zip(lst,lst2)),columns=['metris','value'])
results_df

,metris,value
0,mae,3.666667
1,rmse,17.666667
2,rsquared,-5.297030


,rkey,value_x,value_y
0,foo,1,5
1,foo,1,8
2,foo,5,5
3,foo,5,8
4,bar,2,6
5,baz,3,7
